In [79]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.stem.snowball import SnowballStemmer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import LabelBinarizer

In [80]:
# import nltk
# nltk.download()

In [81]:
train = pd.read_csv(".\\data\\train.csv", encoding='utf=8')
test = pd.read_csv(".\\data\\test.csv", encoding='utf=8')
print train.shape
print test.shape

(38932, 5)
(29404, 4)


In [82]:
train_len = train.shape[0]
print train_len

38932


In [83]:
train_labels = train["Is_Response"]
train.drop("Is_Response", axis=1, inplace=True)
print train.shape
print train.columns

(38932, 4)
Index([u'User_ID', u'Description', u'Browser_Used', u'Device_Used'], dtype='object')


In [84]:
total_data = train.append(test)
print total_data.shape

(68336, 4)


In [85]:
cols = ['Browser_Used', 'Device_Used']
for c in cols:
    lb = LabelEncoder()
    total_data[c] = lb.fit_transform(total_data[c])
    print lb.classes_

[u'Chrome' u'Edge' u'Firefox' u'Google Chrome' u'IE' u'Internet Explorer'
 u'InternetExplorer' u'Mozilla' u'Mozilla Firefox' u'Opera' u'Safari']
[u'Desktop' u'Mobile' u'Tablet']


In [56]:
# stemmer = PorterStemmer()
# # stop_words = set(stopwords.words('english'))
# for i,r in total_data.iterrows():
#     wt = word_tokenize(r[1])
#     filtered_desc = []
#     for w in wt:
#         filtered_desc.append(stemmer.stem(w))
#     print i,
#     total_data.loc[i,"Description"] = " ".join(filtered_desc)

In [86]:
stemmed_df = pd.read_csv("stemmed.csv", encoding='utf=8')
total_data["Description"] = stemmed_df["Description"]
# total_data.to_csv("stemmed.csv", index=False,encoding ='utf-8')

In [87]:
train_new = total_data.iloc[:train_len,:]
submission = total_data.iloc[train_len:,:]
submission_user_id = submission["User_ID"]
train_new.drop("User_ID", axis=1, inplace=True)
submission.drop("User_ID", axis=1, inplace=True)
print train_new.shape
print submission.shape

D:\sasnjeev\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(38932, 3)
(29404, 3)


D:\sasnjeev\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [88]:
print train_new.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_new,train_labels, test_size=0.33, random_state=2017)

(38932, 3)


In [89]:
print X_train.shape
print X_train.columns
print X_test.shape
print X_test.columns
print y_train.shape
# print y_train.columns
print y_test.shape

(26084, 3)
Index([u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(12848, 3)
Index([u'Description', u'Browser_Used', u'Device_Used'], dtype='object')
(26084L,)
(12848L,)


In [110]:
cv = CountVectorizer(stop_words='english', min_df=5, ngram_range=(1,2), max_features=1000)
cv.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist()+submission.Description.values.tolist())
tfidf = TfidfVectorizer(stop_words='english', min_df=5, ngram_range=(1,2), max_features=1000)
tfidf.fit(X_train.Description.values.tolist()+X_test.Description.values.tolist()+submission.Description.values.tolist())

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=5,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [111]:
X_train_desc_vec = cv.transform(X_train.Description.values.tolist())
X_test_desc_vec = cv.transform(X_test.Description.values.tolist())
submission_desc_vec = cv.transform(submission.Description.values.tolist())

In [112]:
X_train_desc_tfidf = tfidf.transform(X_train.Description.values.tolist())
X_test_desc_tfidf = tfidf.transform(X_test.Description.values.tolist())
submission_desc_tfidf = tfidf.transform(submission.Description.values.tolist())

In [113]:
lb_labels = LabelBinarizer()
# print y_train[:5]
y_train = lb_labels.fit_transform(y_train)
y_test = lb_labels.transform(y_test)
print lb_labels.classes_

[0 1]


In [114]:
xtrain = hstack([csr_matrix(X_train[cols].astype(float).values), X_train_desc_vec, X_train_desc_tfidf])
xtest = hstack([csr_matrix(X_test[cols].astype(float).values), X_test_desc_vec, X_test_desc_tfidf])
xsubmission = hstack([csr_matrix(submission[cols].astype(float).values), submission_desc_vec, submission_desc_tfidf])

In [115]:
dtrain = xgb.DMatrix(xtrain, label=y_train)
dtest = xgb.DMatrix(xtest, label=y_test)
dsubmission = xgb.DMatrix(xsubmission)

In [116]:
params = {'objective': 'binary:logistic',
          'booster': 'gbtree',
          'eval_metric': 'error',
          'nthread': 4,
          'silent': 1,
          'max_depth': 6,
          'subsample': 0.9,
          'min_child_weight': 1,
          "colsample_bytree": 0.9,
          'eta': 0.05,
          'seed': 2017}
watchlist = [(dtrain, 'train'), (dtest, 'test')]
num_rounds = 250
clf_xgb = xgb.train(params, dtrain, num_rounds, verbose_eval=10, evals=watchlist, early_stopping_rounds=10)

[0]	train-error:0.206563	test-error:0.229452
Multiple eval metrics have been passed: 'test-error' will be used for early stopping.

Will train until test-error hasn't improved in 10 rounds.
[10]	train-error:0.186436	test-error:0.20797
[20]	train-error:0.173133	test-error:0.198241
[30]	train-error:0.161287	test-error:0.188278
[40]	train-error:0.150859	test-error:0.180806
[50]	train-error:0.144725	test-error:0.175202
[60]	train-error:0.136367	test-error:0.170922
[70]	train-error:0.130271	test-error:0.165862
[80]	train-error:0.123869	test-error:0.161037
[90]	train-error:0.11992	test-error:0.1566
[100]	train-error:0.114936	test-error:0.155044
[110]	train-error:0.110106	test-error:0.151385
[120]	train-error:0.106195	test-error:0.14835
[130]	train-error:0.103052	test-error:0.146482
[140]	train-error:0.099141	test-error:0.144225
[150]	train-error:0.096266	test-error:0.14259
[160]	train-error:0.093084	test-error:0.141267
[170]	train-error:0.08971	test-error:0.1401
[180]	train-error:0.08741	tes

In [117]:
pred = clf_xgb.predict(dtest)
print pred[:5]
print y_test[:5]
pred_ = pred>0.5
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, pred_)
print score

[ 0.42097336  0.10202809  0.09633233  0.14175852  0.54209566]
[[1]
 [0]
 [0]
 [1]
 [0]]
0.862546699875


In [118]:
submit_pred = clf_xgb.predict(dsubmission)
print submit_pred.shape

(29404L,)


In [119]:
submit_pred = lb_labels.inverse_transform(submit_pred)
submit_df = pd.concat([submission_user_id, pd.DataFrame(submit_pred)], axis=1)
submit_df.columns = ["User_ID", "Is_Response"]
print submit_df.shape

(29404, 2)


In [120]:
submit_df.head(5)
submit_df.to_csv("submission.csv", index=False)